# CSM Tutorial

## Spring Elements Problems

In this tutorial, we will show the typical workflow of solving a spring elements problem using the CSM (Computational Solid Mechanics) framework. As we will see, the framework allows to easily define the problem and solve it quickly in very few lines of code.

### Problem Description

The spring element is a simple one-dimensional element that connects two nodes. The spring has a stiffness `k` and can be compressed or stretched. As the spring element has only two degrees of freedom (DOFs), one at each node, the element stiffness matrix is a 2x2 matrix defined as:

$$
k_{el} = \begin{bmatrix}
k & -k \\
-k & k
\end{bmatrix}
$$

Therefore, for a problem consisting of a system of spring elements with `n` nodes, the global stiffness matrix `K` will be a `n x n` matrix. The global stiffness matrix is assembled from the element stiffness matrices and has to be symmetric and positive definite. Once the global stiffness matrix is assembled, the following system of equations is obtained:

$$
K \cdot U = F
$$

where `U` is the global nodal displacement vector and `F` is the global nodal force vector.

After imposing the boundary conditions, the system of equations can be solved for the unknown nodal displacements. The element forces can then be computed using the element stiffness matrix and the nodal displacements.

### Example Problem

In this example, we will solve a simple problem consisting of two spring elements connected in series. The first element has a stiffness of `k1 = 100 kN/m` and the second element has a stiffness of `k2 = 200 kN/m`. The system is subjected to a total force of `F = 15 kN` at the free end of the second element. The first node is fixed, and the second node is free.

### Solution Steps using CSM
1. **Import Required Libraries**: first, we need to import the SpringProblem class from the CSM framework.

In [1]:
# Import the necessary libraries
from CSM import SpringProblem

2. **Define the Problem**: we define the problem parameters, including the nodes, elements and stiffnesses. The nodes are defined as a list of tuples, where each tuple contains the coordinates of the node. The elements are defined as a list of tuples, where each tuple contains the indices of the nodes that form the element. The stiffnesses are defined as a list of floats, where each float corresponds to the stiffness of the respective element.

In [2]:
# Define the parameters for the spring problem
nodes = [(0, 0), (0, 1), (0, 2)] # 3 nodes in a vertical line
elements = [(0, 1), (1, 2)] # 2 elements connecting the nodes
k = [100, 200]

3. **Create the Problem Instance**: we create an instance of the SpringProblem class and set the problem parameters.

In [3]:
# Create an instance of the SpringProblem class
# with the defined nodes, elements, and spring constants
spring = SpringProblem(nodes, elements, k)

4. **Assemble the Global Stiffness Matrix**: we call the `assemble_global_stiffness` method to assemble the global stiffness matrix. As mentioned earlier, the global stiffness matrix is obtained by assembling the element stiffness matrices.

In [4]:
K = spring.assemble_global_stiffness()
# Print the global stiffness matrix
print("Global stiffness matrix:")
print(K)

Global stiffness matrix:
[[ 100. -100.    0.]
 [-100.  300. -200.]
 [   0. -200.  200.]]


If we need some information about the SpringProblem class or some of its methods, we can use the `help` function to get a list of all the methods and their docstrings. For example, to get help on the `assemble_global_stiffness` method, we can do:

In [5]:
help(spring.assemble_global_stiffness)

Help on method assemble_global_stiffness in module CSM.spring:

assemble_global_stiffness() method of CSM.spring.SpringProblem instance
    Assemble the global stiffness matrix from local stiffness matrices.
    
    Returns:
    np.ndarray: Global stiffness matrix.



5. **Apply Boundary Conditions** (*optional*): we apply the boundary conditions by setting the fixed node's displacement to zero. To do this, we can use the `set_external_constraints` method, which takes a list as input. Each element of the list is the index of the node that is fixed. In this case, we will set the first node (index 0) as fixed. The method will also partition the global stiffness matrix `K`, removing the rows and columns corresponding to the fixed node. The resulting matrix will be a reduced matrix that can be used to solve the system of equations.

In [6]:
Reduced_K = spring.set_external_constraints([0])
# Print the reduced stiffness matrix
print("Reduced stiffness matrix:")
print(Reduced_K)

Reduced stiffness matrix:
[[ 300. -200.]
 [-200.  200.]]


6. **Solve the System of Equations**: we call the `solve` method to solve the system of equations for the unknown nodal displacements. The method returns the global nodal displacement vector `U`, which contains the displacements of all nodes in the system. In doing so, the method will also automatically call the `set_external_constraints` method to apply the boundary conditions and therefore we don't need to call it explicitly as we did in the previous step just to show how it works. The `solve` method takes as input the list of external forces `F` and the list of fixed nodes.

In [7]:
F = [0, 15] # Define the force vector
fixed_nodes = [0] # Define the fixed nodes
# Solve the system with the given force vector and fixed nodes
U = spring.solve(F, fixed_nodes)
# Print the displacement vector
print("Displacement vector (m):")
print(U)

Displacement vector (m):
[0.    0.15  0.225]


We can obtain the displacement of a specific node by calling the `get_displacement` method, which takes as input the index of the node. The method returns the displacement of the node:

In [8]:
# Get the displacement of the second node
u2 = spring.get_displacement(1)
# Print the displacement of the second node
print("Displacement of node 1 (m):")
print(u2)

Displacement of node 1 (m):
0.15000000000000002


7. **Compute the Reaction Forces**: we can get the global force vector `F` by calling the `get_reaction_forces` method. This method returns the global force vector `F`, which contains the forces acting on all nodes in the system.

In [9]:
# Get the reaction forces
F = spring.get_reaction_forces()
# Print the reaction forces
print("Reaction forces (kN):")
print(F)

Reaction forces (kN):
[-1.5000000e+01  4.4408921e-16  1.5000000e+01]


8. **Compute the Element Forces**: finally, we can get the element forces by calling the `get_element_force` method. This method takes as input the index of the element and returns the vector of forces acting on the element.

In [10]:
for i in range(len(elements)):
    # Get the element forces for the i-th element
    f = spring.get_element_force(i)
    # Print the element forces
    print(f"Element {i} forces (kN):")
    print(f)


Element 0 forces (kN):
[-15.  15.]
Element 1 forces (kN):
[-15.  15.]
